# CECRL Prompt Testing Framework

This notebook tests v2.0.0 vs v2.1.0 prompt changes for CECRL document extraction.

## Test Cases:
1. **US Passport (Venezuela birth)**: Fix nationality confusion 
2. **Colombian Cédula**: Fix firstName/lastName swapping

## Usage:
- **Change S3 paths** in first cell if needed
- **Run cells sequentially** or individual tests
- **Check outputs/** directory for detailed results

In [ ]:
# =============================================================================
# SETUP AND CONFIGURATION
# =============================================================================

# Import all testing modules
from config import *
from engine import *
from run import *

print("✅ All modules loaded")
print(f"📋 Configured documents: {list(TEST_DOCUMENTS.keys())}")
print(f"🎯 Model: {BEDROCK_MODEL}")

## 📋 View Test Documents

See what documents are configured for testing:

In [3]:
# Show all configured test documents
list_documents()

📋 Configured Test Documents:
----------------------------------------
🔑 us_passport_venezuela
   Name: US Passport - Venezuela Birth
   Path: s3://par-servicios-poc-qa-filling-desk/par-servicios-poc/CECRL/984174004/_2022-01-06.pdf
   Expected fixes: {'nationality': 'Venezuela'}

🔑 colombian_cedula
   Name: Colombian Cédula - Name Swap
   Path: s3://par-servicios-poc-qa-filling-desk/par-servicios-poc/CECRL/900397317/19200964_2020-02-29.pdf
   Expected fixes: {'firstName': 'Luis Ignacio', 'lastName': 'Hernández Díaz'}



## 🔧 Setup AWS Clients

Initialize Bedrock and S3 clients:

In [ ]:
# Setup AWS clients
bedrock_client, s3_client = setup_aws_clients()
print("✅ AWS clients ready")

## 🧪 Quick Test - Single Document

Test the first document quickly:

In [ ]:
# Quick test on first document
quick_test()

## 🎯 Test Specific Document

Test a specific document by key:

In [ ]:
# Test US passport document
run_single_document_test("us_passport_venezuela")

In [ ]:
# Test Colombian cédula document
run_single_document_test("colombian_cedula")

## 🚀 Run Complete Test Suite

Test all documents with both prompt versions:

In [ ]:
# Run all tests
run_all_tests()

## 📊 View Latest Results

Load and display the latest comparison report:

In [ ]:
# Load latest comparison report
report_path = COMPARISON_DIR / "comparison_report.json"
if report_path.exists():
    with open(report_path) as f:
        report = json.load(f)

    print_summary(report)
else:
    print("❌ No comparison report found. Run tests first.")

## 🔍 Detailed Field Analysis

Examine specific field changes in detail:

In [ ]:
# Detailed analysis of field changes
def show_detailed_changes(document_key: str):
    """Show detailed field changes for a specific document."""
    report_path = COMPARISON_DIR / "comparison_report.json"
    if not report_path.exists():
        print("❌ No comparison report found. Run tests first.")
        return

    with open(report_path) as f:
        report = json.load(f)

    if document_key not in report["document_comparisons"]:
        print(f"❌ Document '{document_key}' not found in report")
        return

    comparison = report["document_comparisons"][document_key]

    print(f"\n🔍 Detailed Analysis: {comparison['document_name']}")
    print("="*50)

    print(f"📄 Description: {comparison['description']}")
    print(f"🎯 Expected fixes: {comparison['expected_fixes']}")

    print("\n📊 Field-by-Field Comparison:")
    print("-"*30)

    for field, change_data in comparison["field_changes"].items():
        if change_data.get("changed", False):
            print(f"🔄 {field}:")
            print(f"   v2.0.0: '{change_data['old_value']}'")
            print(f"   v2.1.0: '{change_data['new_value']}'")
            print()
        else:
            print(f"✅ {field}: '{change_data['value']}' (unchanged)")

# Example usage:
show_detailed_changes("us_passport_venezuela")

## 📁 File Outputs

All test results are saved to:

- `outputs/batch_results.json` - Raw test results
- `outputs/comparison/comparison_report.json` - Formatted comparison report  
- `outputs/before/` - v2.0.0 results by document
- `outputs/after/` - v2.1.0 results by document
- `outputs/test_log.txt` - Execution logs

## 🔧 Adding New Test Documents

To test additional documents, modify `TEST_DOCUMENTS` in `config.py`:

```python
TEST_DOCUMENTS["new_document"] = {
    "name": "Description",
    "description": "What should be fixed",
    "s3_path": "s3://bucket/path/file.pdf", 
    "expected_fixes": {"field": "expected_value"}
}
```